In [6]:
import numpy as np
from astropy.io import fits
from scipy import ndimage
from scipy.signal import correlate
from scipy.ndimage import fourier_shift
import os
from pathlib import Path

def cross_correlation_shift(reference, target):
    """
    Calculate the shift between reference and target images using cross-correlation.
    Returns subpixel shift values (dy, dx).
    """
    # Compute cross-correlation
    correlation = correlate(reference, target, mode='same', method='fft')
    
    # Find the peak of correlation
    peak = np.unravel_index(np.argmax(correlation), correlation.shape)
    
    # Calculate shift (center of image is zero shift)
    center = np.array(reference.shape) // 2
    shift = np.array(peak) - center
    
    # Refine to subpixel precision using parabolic interpolation
    def subpixel_peak(corr, peak_idx):
        """Refine peak position to subpixel accuracy"""
        y, x = peak_idx
        
        # Check boundaries
        if y <= 0 or y >= corr.shape[0]-1 or x <= 0 or x >= corr.shape[1]-1:
            return peak_idx
        
        # Parabolic interpolation in y direction
        dy = 0
        denom_y = 2 * (corr[y-1, x] - 2*corr[y, x] + corr[y+1, x])
        if denom_y != 0:
            dy = (corr[y-1, x] - corr[y+1, x]) / denom_y
        
        # Parabolic interpolation in x direction
        dx = 0
        denom_x = 2 * (corr[y, x-1] - 2*corr[y, x] + corr[y, x+1])
        if denom_x != 0:
            dx = (corr[y, x-1] - corr[y, x+1]) / denom_x
        
        return (y + dy, x + dx)
    
    refined_peak = subpixel_peak(correlation, peak)
    refined_shift = np.array(refined_peak) - center
    
    return refined_shift

def apply_shift(image, shift):
    """
    Apply subpixel shift to image using Fourier shift theorem.
    shift: (dy, dx) tuple
    """
    # Fourier shift for subpixel precision
    shifted = fourier_shift(np.fft.fftn(image), shift)
    shifted = np.fft.ifftn(shifted).real
    
    return shifted

def align_images(reference_path, target_paths, output_dir):
    """
    Align multiple images to a reference using cross-correlation.
    
    Parameters:
    -----------
    reference_path : str
        Path to reference FITS file
    target_paths : list
        List of paths to target FITS files to align
    output_dir : str
        Directory to save aligned images
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Load reference image
    print(f"Loading reference image: {reference_path}")
    with fits.open(reference_path) as hdul:
        reference_data = hdul[0].data.astype(np.float64)
        reference_header = hdul[0].header.copy()
    
    # Save reference image (no alignment needed)
    ref_filename = Path(reference_path).name.replace('normalized', 'aligned')
    ref_output_path = os.path.join(output_dir, ref_filename)
    fits.writeto(ref_output_path, reference_data, reference_header, overwrite=True)
    print(f"Reference saved: {ref_output_path}")
    print()
    
    # Process each target image
    for target_path in target_paths:
        print(f"Processing: {target_path}")
        
        # Load target image
        with fits.open(target_path) as hdul:
            target_data = hdul[0].data.astype(np.float64)
            target_header = hdul[0].header.copy()
        
        # Calculate shift
        print("  Calculating cross-correlation shift...")
        shift = cross_correlation_shift(reference_data, target_data)
        print(f"  Detected shift: dy={shift[0]:.3f}, dx={shift[1]:.3f} pixels")
        
        # Apply shift
        print("  Applying subpixel shift...")
        aligned_data = apply_shift(target_data, shift)
        
        # Update header with alignment info
        target_header['ALIGNED'] = (True, 'Image aligned using cross-correlation')
        target_header['ALIGNREF'] = (Path(reference_path).name, 'Reference image for alignment')
        target_header['SHIFT_Y'] = (float(shift[0]), 'Y shift in pixels')
        target_header['SHIFT_X'] = (float(shift[1]), 'X shift in pixels')
        
        # Save aligned image
        output_filename = Path(target_path).name.replace('normalized', 'aligned')
        output_path = os.path.join(output_dir, output_filename)
        fits.writeto(output_path, aligned_data, target_header, overwrite=True)
        print(f"  Saved: {output_path}")
        print()

def main():
    # Define paths
    input_dir = "/home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/normalized"
    output_dir = "/home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/second_alignment_v1"
    
    # Define filters
    filters = ['clear', 'g', 'r', 'i']
    reference_filter = 'r'
    
    # Construct file paths
    file_paths = {}
    for filt in filters:
        filename = f"normalized_NGC1365_{filt}_stacked_sigma3.0.fits"
        file_paths[filt] = os.path.join(input_dir, filename)
    
    # Verify all files exist
    print("Checking input files...")
    for filt, path in file_paths.items():
        if not os.path.exists(path):
            print(f"ERROR: File not found: {path}")
            return
        else:
            print(f"  Found: {filt} - {path}")
    print()
    
    # Reference image
    reference_path = file_paths[reference_filter]
    
    # Target images (all except reference)
    target_paths = [file_paths[filt] for filt in filters if filt != reference_filter]
    
    # Perform alignment
    print("="*70)
    print("Starting alignment process...")
    print(f"Reference filter: {reference_filter}")
    print("="*70)
    print()
    
    align_images(reference_path, target_paths, output_dir)
    
    print("="*70)
    print("Alignment complete!")
    print(f"Aligned images saved to: {output_dir}")
    print("="*70)

if __name__ == "__main__":
    main()

Checking input files...
  Found: clear - /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/normalized/normalized_NGC1365_clear_stacked_sigma3.0.fits
  Found: g - /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/normalized/normalized_NGC1365_g_stacked_sigma3.0.fits
  Found: r - /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/normalized/normalized_NGC1365_r_stacked_sigma3.0.fits
  Found: i - /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/normalized/normalized_NGC1365_i_stacked_sigma3.0.fits

Starting alignment process...
Reference filter: r

Loading reference image: /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/normalized/normalized_NGC1365_r_stacked_sigma3.0.fits
Reference saved: /home/devika/PhD/S2/Obs_Ast

  Saved: /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/second_alignment/aligned_NGC1365_clear_stacked_sigma3.0.fits

Processing: /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/normalized/normalized_NGC1365_g_stacked_sigma3.0.fits
  Calculating cross-correlation shift...
  Detected shift: dy=-2.486, dx=-16.461 pixels
  Applying subpixel shift...
  Saved: /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/second_alignment/aligned_NGC1365_g_stacked_sigma3.0.fits

Processing: /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/normalized/normalized_NGC1365_i_stacked_sigma3.0.fits
  Calculating cross-correlation shift...
  Detected shift: dy=16.820, dx=14.604 pixels
  Applying subpixel shift...
  Saved: /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reducti

In [2]:
import numpy as np
from astropy.io import fits
from scipy.ndimage import shift
from scipy.fft import fft2, ifft2, fftshift
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
import time
warnings.filterwarnings('ignore')

# Configuration
BASE_PATH = "/home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED"
OUTPUT_PATH = "/home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/aligned"

# Filters and object
FILTERS = ['clear', 'g', 'r', 'i']
OBJECT_NAME = "NGC1365"
REFERENCE_FILTER = 'r'

# All stretch methods to align
STRETCH_METHODS = [
    'normalized',
    'log',
    'hyperbolic',
    'asinh',
    'gamma',
    'adaptive_asinh'
]

# Cross-correlation parameters (optimized for speed)
CORRELATION_PARAMS = {
    'subsample': 4,
    'edge_crop': 100,
    'use_central_region': True,
    'central_size': 800,
}

# Use parabolic refinement (much faster than optimize)
USE_SUBPIXEL_REFINEMENT = True


def load_stretched_image(filter_name, method):
    """Load stretched FITS image"""
    if method == 'normalized':
        subdir = 'normalized'
        filename = f"normalized_{OBJECT_NAME}_{filter_name}_stacked_sigma3.0.fits"
    else:
        subdir = 'stretched'
        filename = f"stretched_{method}_{OBJECT_NAME}_{filter_name}.fits"
    
    filepath = Path(BASE_PATH) / subdir / filename
    
    if not filepath.exists():
        raise FileNotFoundError(f"File not found: {filepath}")
    
    with fits.open(filepath) as hdul:
        data = hdul[0].data.astype(np.float64)
        header = hdul[0].header
    
    return data, header


def prepare_for_correlation(image, subsample=1, edge_crop=50, 
                           use_central=False, central_size=1024):
    """Prepare image for cross-correlation"""
    image = np.nan_to_num(image, nan=0.0)
    
    if edge_crop > 0:
        image = image[edge_crop:-edge_crop, edge_crop:-edge_crop]
    
    if use_central and image.shape[0] > central_size:
        cy, cx = image.shape[0] // 2, image.shape[1] // 2
        half_size = central_size // 2
        image = image[cy-half_size:cy+half_size, cx-half_size:cx+half_size]
    
    if subsample > 1:
        image = image[::subsample, ::subsample]
    
    # Normalize
    image = image - np.mean(image)
    std = np.std(image)
    if std > 0:
        image = image / std
    
    return image


def cross_correlate_images(ref_image, target_image):
    """
    Compute cross-correlation using FFT (very fast)
    """
    # Pad to avoid edge effects
    pad_y = ref_image.shape[0] // 4
    pad_x = ref_image.shape[1] // 4
    
    ref_padded = np.pad(ref_image, ((pad_y, pad_y), (pad_x, pad_x)), mode='constant')
    target_padded = np.pad(target_image, ((pad_y, pad_y), (pad_x, pad_x)), mode='constant')
    
    # FFT correlation
    ref_fft = fft2(ref_padded)
    target_fft = fft2(target_padded)
    correlation = fftshift(ifft2(ref_fft * np.conj(target_fft)).real)
    
    # Find peak
    peak_pos = np.unravel_index(np.argmax(correlation), correlation.shape)
    center_y, center_x = np.array(correlation.shape) // 2
    dy = peak_pos[0] - center_y
    dx = peak_pos[1] - center_x
    
    return dy, dx, correlation


def refine_shift_parabolic(correlation, dy, dx):
    """
    Refine shift to sub-pixel accuracy using parabolic fit
    """
    ny, nx = correlation.shape
    center_y, center_x = ny // 2, nx // 2
    peak_y = center_y + dy
    peak_x = center_x + dx
    
    # Check bounds
    if peak_y <= 0 or peak_y >= ny-1 or peak_x <= 0 or peak_x >= nx-1:
        return dy, dx
    
    # Parabolic fit in X
    x_vals = correlation[peak_y, peak_x-1:peak_x+2]
    if len(x_vals) == 3:
        denom = x_vals[0] - 2*x_vals[1] + x_vals[2]
        if abs(denom) > 1e-10:
            dx_sub = 0.5 * (x_vals[0] - x_vals[2]) / denom
            dx = dx + dx_sub
    
    # Parabolic fit in Y
    y_vals = correlation[peak_y-1:peak_y+2, peak_x]
    if len(y_vals) == 3:
        denom = y_vals[0] - 2*y_vals[1] + y_vals[2]
        if abs(denom) > 1e-10:
            dy_sub = 0.5 * (y_vals[0] - y_vals[2]) / denom
            dy = dy + dy_sub
    
    return dy, dx


def apply_shift(image, dy, dx, subsample=1):
    """Apply shift to full resolution image"""
    full_dy = dy * subsample
    full_dx = dx * subsample
    shifted = shift(image, [full_dy, full_dx], order=3, mode='constant', cval=0)
    return shifted, full_dy, full_dx


def align_filter(ref_data, target_data):
    """Align target to reference using cross-correlation"""
    start = time.time()
    
    # Prepare images
    ref_prep = prepare_for_correlation(
        ref_data,
        subsample=CORRELATION_PARAMS['subsample'],
        edge_crop=CORRELATION_PARAMS['edge_crop'],
        use_central=CORRELATION_PARAMS['use_central_region'],
        central_size=CORRELATION_PARAMS['central_size']
    )
    
    target_prep = prepare_for_correlation(
        target_data,
        subsample=CORRELATION_PARAMS['subsample'],
        edge_crop=CORRELATION_PARAMS['edge_crop'],
        use_central=CORRELATION_PARAMS['use_central_region'],
        central_size=CORRELATION_PARAMS['central_size']
    )
    
    # Cross-correlate
    dy, dx, correlation = cross_correlate_images(ref_prep, target_prep)
    
    # Sub-pixel refinement
    if USE_SUBPIXEL_REFINEMENT:
        dy, dx = refine_shift_parabolic(correlation, dy, dx)
    
    # Apply shift
    aligned, full_dy, full_dx = apply_shift(
        target_data, dy, dx,
        subsample=CORRELATION_PARAMS['subsample']
    )
    
    # Calculate correlation coefficient
    ref_norm = (ref_data - np.mean(ref_data)) / (np.std(ref_data) + 1e-10)
    aligned_norm = (aligned - np.mean(aligned)) / (np.std(aligned) + 1e-10)
    corr = np.sum(ref_norm * aligned_norm) / ref_norm.size
    
    elapsed = time.time() - start
    
    return aligned, full_dy, full_dx, corr, elapsed


def create_diagnostic(ref_data, original_data, aligned_data, 
                     filter_name, method, dy, dx, corr, output_path):
    """Create diagnostic plots"""
    fig = plt.figure(figsize=(20, 10))
    gs = fig.add_gridspec(2, 4, hspace=0.25, wspace=0.3)
    
    # Determine display range
    if method in ['normalized', 'log', 'gamma']:
        vmin, vmax = 0, 1
    else:
        vmin, vmax = 0, 0.8
    
    # Row 1: Full images
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.imshow(ref_data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.set_title(f'Reference ({REFERENCE_FILTER})', fontsize=11, fontweight='bold')
    ax1.axis('off')
    
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.imshow(original_data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax2.set_title(f'Original ({filter_name})', fontsize=11, fontweight='bold')
    ax2.axis('off')
    
    ax3 = fig.add_subplot(gs[0, 2])
    ax3.imshow(aligned_data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax3.set_title(f'Aligned ({filter_name})', fontsize=11, fontweight='bold')
    ax3.axis('off')
    
    ax4 = fig.add_subplot(gs[0, 3])
    diff = ref_data - aligned_data
    vmax_diff = np.percentile(np.abs(diff), 98)
    ax4.imshow(diff, cmap='RdBu_r', origin='lower', vmin=-vmax_diff, vmax=vmax_diff)
    ax4.set_title('Difference', fontsize=11, fontweight='bold')
    ax4.axis('off')
    
    # Row 2: Zoomed views
    cy, cx = ref_data.shape[0] // 2, ref_data.shape[1] // 2
    zoom = 400
    
    ax5 = fig.add_subplot(gs[1, 0])
    ax5.imshow(ref_data[cy-zoom:cy+zoom, cx-zoom:cx+zoom],
               cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax5.set_title('Reference (zoom)', fontsize=11)
    ax5.axis('off')
    
    ax6 = fig.add_subplot(gs[1, 1])
    ax6.imshow(original_data[cy-zoom:cy+zoom, cx-zoom:cx+zoom],
               cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax6.set_title('Original (zoom)', fontsize=11)
    ax6.axis('off')
    
    ax7 = fig.add_subplot(gs[1, 2])
    ax7.imshow(aligned_data[cy-zoom:cy+zoom, cx-zoom:cx+zoom],
               cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax7.set_title('Aligned (zoom)', fontsize=11)
    ax7.axis('off')
    
    # Statistics
    ax8 = fig.add_subplot(gs[1, 3])
    ax8.axis('off')
    
    total = np.sqrt(dy**2 + dx**2)
    status = '✓ EXCELLENT' if corr > 0.95 else '✓ GOOD' if corr > 0.90 else '⚠ CHECK'
    
    stats = f"""
ALIGNMENT STATISTICS
{'='*35}

Method: {method}
Filter: {filter_name} → {REFERENCE_FILTER}

Shift Applied:
  dy = {dy:.3f} px
  dx = {dx:.3f} px
  Total = {total:.3f} px

Correlation: {corr:.6f}

Status: {status}
    """
    
    ax8.text(0.1, 0.95, stats, transform=ax8.transAxes,
             fontsize=10, verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.suptitle(f'{OBJECT_NAME} - {method.upper()} - {filter_name} → {REFERENCE_FILTER}',
                fontsize=14, fontweight='bold')
    
    # Save
    outfile = Path(output_path) / f"alignment_{method}_{OBJECT_NAME}_{filter_name}.png"
    plt.savefig(outfile, dpi=120, bbox_inches='tight')
    plt.close()


def process_method(method):
    """Process all filters for one stretch method"""
    print(f"\n{'='*80}")
    print(f"PROCESSING: {method.upper()}")
    print(f"{'='*80}")
    
    method_dir = Path(OUTPUT_PATH) / method
    method_dir.mkdir(parents=True, exist_ok=True)
    
    # Load reference
    print(f"\n  Loading reference: {REFERENCE_FILTER}")
    try:
        ref_data, ref_header = load_stretched_image(REFERENCE_FILTER, method)
        print(f"    Shape: {ref_data.shape}, Range: [{np.nanmin(ref_data):.4f}, {np.nanmax(ref_data):.4f}]")
    except FileNotFoundError as e:
        print(f"    ✗ {e}")
        return {}
    
    # Save reference
    ref_file = method_dir / f"aligned_{method}_{OBJECT_NAME}_{REFERENCE_FILTER}.fits"
    ref_header['ALIGNED'] = 'REFERENCE'
    ref_header['ALIGNMET'] = 'CROSS-CORRELATION'
    ref_header['STRETCH'] = method
    fits.writeto(ref_file, ref_data.astype(np.float32), ref_header, overwrite=True)
    print(f"    Saved: {ref_file.name}")
    
    # Align other filters
    results = {}
    
    for filt in FILTERS:
        if filt == REFERENCE_FILTER:
            continue
        
        print(f"\n  Processing: {filt}")
        
        try:
            target_data, target_header = load_stretched_image(filt, method)
            print(f"    Shape: {target_data.shape}, Range: [{np.nanmin(target_data):.4f}, {np.nanmax(target_data):.4f}]")
            
            original = target_data.copy()
            
            # Align
            print(f"    Aligning...")
            aligned, dy, dx, corr, elapsed = align_filter(ref_data, target_data)
            
            total = np.sqrt(dy**2 + dx**2)
            print(f"    Shift: dy={dy:.3f}, dx={dx:.3f}, total={total:.3f} px")
            print(f"    Correlation: {corr:.6f}")
            print(f"    Time: {elapsed:.2f}s")
            
            # Save
            out_file = method_dir / f"aligned_{method}_{OBJECT_NAME}_{filt}.fits"
            target_header['ALIGNED'] = 'YES'
            target_header['ALIGNMET'] = 'CROSS-CORRELATION'
            target_header['ALIGNREF'] = REFERENCE_FILTER
            target_header['STRETCH'] = method
            target_header['SHIFTDY'] = (dy, 'Y shift (pixels)')
            target_header['SHIFTDX'] = (dx, 'X shift (pixels)')
            target_header['SHIFTTOT'] = (total, 'Total shift (pixels)')
            target_header['CORR'] = (corr, 'Correlation coefficient')
            
            fits.writeto(out_file, aligned.astype(np.float32), target_header, overwrite=True)
            print(f"    Saved: {out_file.name}")
            
            # Diagnostic
            print(f"    Creating diagnostic...")
            create_diagnostic(ref_data, original, aligned, filt, method, dy, dx, corr, method_dir)
            
            results[filt] = {'dy': dy, 'dx': dx, 'total': total, 'corr': corr}
            print(f"    ✓ Complete")
            
        except Exception as e:
            print(f"    ✗ Error: {e}")
            continue
    
    return results


def main():
    """Main function"""
    print("="*80)
    print(f"MULTI-STRETCH ALIGNMENT - {OBJECT_NAME}")
    print("="*80)
    print(f"\nInput: {BASE_PATH}")
    print(f"Output: {OUTPUT_PATH}")
    print(f"Reference: {REFERENCE_FILTER}")
    print(f"Methods: {len(STRETCH_METHODS)}")
    print(f"Filters: {[f for f in FILTERS if f != REFERENCE_FILTER]}")
    
    Path(OUTPUT_PATH).mkdir(parents=True, exist_ok=True)
    
    all_results = {}
    total_start = time.time()
    
    for method in STRETCH_METHODS:
        results = process_method(method)
        if results:
            all_results[method] = results
    
    total_elapsed = time.time() - total_start
    
    # Summary
    print("\n" + "="*80)
    print("SUMMARY")
    print("="*80)
    
    for method in STRETCH_METHODS:
        if method not in all_results:
            print(f"\n{method.upper()}: No results")
            continue
        
        print(f"\n{method.upper()}:")
        print(f"  Reference: {REFERENCE_FILTER}")
        print(f"\n  Filter    Shift-Y   Shift-X   Total     Correlation")
        print("  " + "-"*60)
        
        for filt, res in all_results[method].items():
            print(f"  {filt:8s} {res['dy']:8.3f}  {res['dx']:8.3f}  "
                  f"{res['total']:8.3f}  {res['corr']:8.6f}")
    
    print("\n" + "="*80)
    print(f"✓ COMPLETE in {total_elapsed/60:.1f} minutes")
    print("="*80)
    print(f"\nOutputs in: {OUTPUT_PATH}/")
    print("\nNext: Review diagnostic plots and proceed with color combination")


if __name__ == "__main__":
    main()

MULTI-STRETCH ALIGNMENT - NGC1365

Input: /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED
Output: /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/aligned
Reference: r
Methods: 6
Filters: ['clear', 'g', 'i']

PROCESSING: NORMALIZED

  Loading reference: r
    Shape: (2048, 2048), Range: [0.0000, 1.0000]
    Saved: aligned_normalized_NGC1365_r.fits

  Processing: clear
    Shape: (2048, 2048), Range: [0.0000, 1.0000]
    Aligning...
    Shift: dy=15.397, dx=-0.157, total=15.398 px
    Correlation: 0.836961
    Time: 0.40s
    Saved: aligned_normalized_NGC1365_clear.fits
    Creating diagnostic...
    ✓ Complete

  Processing: g
    Shape: (2048, 2048), Range: [0.0000, 1.0000]
    Aligning...
    Shift: dy=-2.922, dx=-15.936, total=16.202 px
    Correlation: 0.680025
    Time: 0.41s
    Saved: aligned_normalized_NGC1365_g.fits
    Creating diagnostic...
    ✓ Complete

  Proce